# Delta Lake 2.3.0 Release

In [3]:
import pyspark
from delta import *
from pyspark.sql import functions as F

builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-332-delta-230/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8c73ba5b-1d54-4188-ac0b-d72a3a31447a;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.3.0/delta-core_2.12-2.3.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;2.3.0!delta-core_2.12.jar (3876ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/2.3.0/delta-storage-2.3.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;2.3.0!delta-storage.jar (375ms)
:: resolution report :: resolve 2256ms :: artifacts dl 4259ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr

23/04/10 07:27:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Convert Iceberg to Delta Lake

TODO: Figure out how to create an Iceberg table

## Delta Lake Shallow Clone

In [4]:
data = [(0, "Bob", 23), (1, "Sue", 25), (2, "Jim", 27)]

In [5]:
df = spark.createDataFrame(data).toDF("id", "name", "age")

In [6]:
df.write.format("delta").save("tmp/my_people")

23/04/10 07:27:54 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [8]:
spark.sql("CREATE TABLE delta.`tmp/my_cloned_people` SHALLOW CLONE delta.`tmp/my_people`")

AnalysisException: Unsupported data source type for direct query on files: delta; line 1 pos 0

## When not matched by source

## CREATE TABLE LIKE

In [10]:
df = spark.createDataFrame(
    [(0, "Bob", "Loblaw", 23), (1, "Sue", "Grafton", None), (2, "Jim", "Carrey", 61)]
).toDF("id", "first_name", "last_name", "age")

In [11]:
df.write.format("delta").saveAsTable(
    "default.famous_people"
)

In [12]:
spark.sql("select * from famous_people").show()

+---+----------+---------+----+
| id|first_name|last_name| age|
+---+----------+---------+----+
|  2|       Jim|   Carrey|  61|
|  0|       Bob|   Loblaw|  23|
|  1|       Sue|  Grafton|null|
+---+----------+---------+----+



In [13]:
spark.sql("CREATE TABLE famous_people_duplicate like famous_people")

DataFrame[]

In [14]:
spark.sql("select * from famous_people_duplicate").show()

+---+----------+---------+---+
| id|first_name|last_name|age|
+---+----------+---------+---+
+---+----------+---------+---+



## Reading change data feed in SQL queries

## Cleanup